<a href="https://colab.research.google.com/github/diegohugo570/backup-python/blob/main/02_LLMs_para_empresas_e_neg%C3%B3cios_Atendimento_e_Suporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto para Atendimento e Suporte

Nosso objetivo com esse projeto é criar um chatbot com RAG que responde com base em documentos reais da empresa (ex.: base de conhecimento, manuais, PDFs).

* Retrieval-Augmented Generation (RAG) é uma técnica que combina modelos de linguagem com mecanismos de recuperação de informações para melhorar a geração de texto.

* O RAG revolucionou a interação, compreensão e geração de linguagem humana pelos sistemas de IA. Tornou os modelos de linguagem mais versáteis e inteligentes, sendo crucial para chatbots sofisticados e ferramentas complexas de criação de conteúdo.

> [ ver slides para mais sobre RAG ]

O LangChain tem vários componentes projetados para ajudar a criar aplicativos de perguntas e respostas e aplicativos RAG de forma mais geral.

Vamos usar primeiro o ipynb no Colab para desenvolver e validar a lógica com LLMs, e só depois adaptar isso para uma interface mais amigável com Streamlit. Isso evita retrabalho, ajuda a testar ideias com rapidez e foca primeiro no que importa: o núcleo funcional, a lógica e conceitos.



## Instalação das bibliotecas


In [ ]:
!pip install langchain langchain-groq langchain_community langchain-huggingface --q
!pip install faiss-cpu sentence-transformers PyMuPDF --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

**Importações**

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

## Carregando a LLM


In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [ ]:
def load_llm(id_model, temperature):
  llm = ChatGroq(
      model = id_model,
      temperature = temperature,
      max_tokens=None,
      timeout=None,
      max_retries=2,
  )
  return llm

In [ ]:
id_model = "llama3-70b-8192" # @param {type: "string"}
temperature = 0.7 # @param {type: "slider", min: 0.1, max: 1.5, step: 0.1}

llm = load_llm(id_model, temperature)

In [ ]:
prompt = "Como alterar minha senha?" # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual prestativo e está respondendo perguntas gerais"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Alterar a senha é uma boa prática de segurança!\n\nPara alterar sua senha, você precisará seguir os seguintes passos:\n\n1. Acesse a página de login do seu aplicativo ou serviço online.\n2. Clique em "Esqueci minha senha" ou "Alterar senha".\n3. Insira seu endereço de e-mail ou nome de usuário associado à sua conta.\n4. Se você estiver usando um aplicativo móvel, você pode receber um código de verificação por SMS ou e-mail. Insira o código para continuar.\n5. Crie uma nova senha forte e única. Certifique-se de que ela atenda aos requisitos de segurança, como ter pelo menos 12 caracteres, incluir letras maiúsculas e minúsculas, números e caracteres especiais.\n6. Insira a nova senha novamente para confirmá-la.\n7. Clique em "Salvar" ou "Alterar senha" para concluir o processo.\n\nLembre-se de que é importante armazenar sua senha em um gerenciador de senhas seguro, para que você possa acessá-la facilmente e mantê-la segura!'

In [ ]:
def show_res(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()
  display(Markdown(res))

show_res(res)

Alterar a senha é uma boa prática de segurança!

Para alterar sua senha, você precisará seguir os seguintes passos:

1. Acesse a página de login do seu aplicativo ou serviço online.
2. Clique em "Esqueci minha senha" ou "Alterar senha".
3. Insira seu endereço de e-mail ou nome de usuário associado à sua conta.
4. Se você estiver usando um aplicativo móvel, você pode receber um código de verificação por SMS ou e-mail. Insira o código para continuar.
5. Crie uma nova senha forte e única. Certifique-se de que ela atenda aos requisitos de segurança, como ter pelo menos 12 caracteres, incluir letras maiúsculas e minúsculas, números e caracteres especiais.
6. Insira a nova senha novamente para confirmá-la.
7. Clique em "Salvar" ou "Alterar senha" para concluir o processo.

Lembre-se de que é importante armazenar sua senha em um gerenciador de senhas seguro, para que você possa acessá-la facilmente e mantê-la segura!

## Definindo o contexto


In [ ]:
context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [ ]:
prompt

"\nComo alterar minha senha?\n\nContexto: \nPara alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.\nPara alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.\n\n"

In [ ]:
res = chain.invoke({"prompt": prompt})
show_res(res)

Para alterar sua senha, você pode seguir os passos abaixo:

**No aplicativo:**

1. Clique no menu "Minha conta".
2. Selecione "Alterar senha".

**No site:**

1. Acesse o menu do topo e clique em "Configurações".
2. Selecione "Alterar senha".

Siguiendo esses passos, você poderá alterar sua senha com facilidade!

## Prompt para RAG


In [ ]:
template_rag = """
Pergunta: {input}
Contexto: {context}
"""

In [ ]:
from langchain.prompts import PromptTemplate

prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['context', 'input'] input_types={} partial_variables={} template='\nPergunta: {input}\nContexto: {context}\n'


## Criação da Chain / Geração

In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

input = "como alterar minhar senha?"

res = chain_rag.invoke({"context": context, "input": input})

show_res(res)

Para alterar sua senha, você pode seguir os seguintes passos:

**No aplicativo:**

1. Clique no menu "Minha conta".
2. Selecione "Alterar senha".

**No site:**

1. Acesse o menu do topo e clique em "Configurações".
2. Em seguida, selecione "Minha conta" e "Alterar senha".

Siga essas etapas para alterar sua senha com segurança!



## Etapas de Indexação

### 1 - Preparar os documentos / Carregar o conteúdo

> Integrações: https://python.langchain.com/docs/integrations/document_loaders/




In [ ]:
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print(f"Arquivo carregado: {file_path}")

Saving manual-safebank.pdf to manual-safebank.pdf
Arquivo carregado: manual-safebank.pdf


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual-safebank.pdf"

loader = PyMuPDFLoader(file_path)
doc = loader.load()
doc[1]

Document(metadata={'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'manual-safebank.pdf', 'file_path': 'manual-safebank.pdf', 'total_pages': 11, 'format': 'PDF 1.4', 'title': 'manual-safebank', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 1}, page_content='Prazo para ativação da conta  \nO processo de verificação leva até 2 dias úteis. Em casos com divergências de dados ou \nimagens ilegíveis, esse prazo pode se estender. O cliente será notificado por e-mail ou \nnotificação no app. \n \nProblemas comuns no cadastro \n●\u200b Documento vencido ou ilegível \n●\u200b Foto do documento fora do enquadramento \n●\u200b Não realização do vídeo de reconhecimento facial \n●\u200b Dados incorretos ou divergentes com a Receita Federal \n \n \n3. Acesso à Conta \nAcesso via site - Acesse o site oficial e clique em "Entrar" no topo da página. Insira seu \nCPF e senha. Caso deseje

In [ ]:
import pprint

pprint.pp(doc[0].metadata)

{'producer': 'Skia/PDF m137 Google Docs Renderer',
 'creator': '',
 'creationdate': '',
 'source': 'manual-safebank.pdf',
 'file_path': 'manual-safebank.pdf',
 'total_pages': 11,
 'format': 'PDF 1.4',
 'title': 'manual-safebank',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'modDate': '',
 'creationDate': '',
 'page': 0}


In [ ]:
len(doc[0].page_content)

1668

In [ ]:
print(doc[0].page_content[:1000])

Manual de Atendimento e Uso – SafeBank 
 
1. Introdução 
Objetivo do Manual  
Este manual tem como objetivo orientar clientes e atendentes sobre o funcionamento 
completo do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, 
até procedimentos de segurança, suporte e resolução de problemas. O documento também 
serve como material de consulta para dúvidas frequentes. 
Visão geral do banco digital  
Nosso banco digital oferece serviços bancários completos de forma 100% online, por meio 
do aplicativo e do site. Entre os principais serviços estão conta digital gratuita, 
transferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e 
muito mais, sempre com foco em praticidade, segurança e transparência. 
Público-alvo  
O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções 
financeiras modernas, acessíveis e com gestão 100% digital.  
 
2. Cadastro e Abertura de Conta 
Requisitos para abertura de conta 

In [ ]:
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()
  content = "\n".join([page.page_content for page in doc])
  return content

In [ ]:
extract_text_pdf(file_path)

'Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções \nfinanceiras modernas, acessíveis e com gestão 100% digital.  \n \n2. Cadastro e Abertura de Conta \nRequisitos para

In [ ]:
docs_path = Path("/content")
pdf_files = [f for f in docs_path.glob("*.pdf")]
print(pdf_files)

[PosixPath('/content/manual-safebank.pdf')]


In [ ]:
loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]
len(pdf_files)

1

### 2 - Divisão em pedaços de texto / Split





In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
chunks = []
for doc in loaded_documents:
  chunks.extend(text_splitter.split_text(doc))

print(f"Número de chunks criados: {len(chunks)}")

Número de chunks criados: 44


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = "sentence-transformers/all-mpnet-base-v2"
#embedding_model = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

input_test = "Um teste apenas"

result = embeddings.embed_query(input_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
len(result)

768

In [ ]:
print(result)

[-0.026895038783550262, -0.011064727790653706, -0.04532000422477722, -0.0013972934102639556, 0.04243597015738487, -0.014201696030795574, 0.023354969918727875, 0.06011558324098587, 0.06152193248271942, 0.006502091884613037, 0.008159181103110313, -0.03056747280061245, 0.002060319297015667, 0.0129615543410182, 0.0042511713691055775, 0.0036631484981626272, -0.026755547150969505, 0.029737001284956932, -0.009770059026777744, -0.04650460556149483, -0.028108958154916763, 0.00016859530296642333, -0.02481103502213955, -0.01182831171900034, 0.0818282887339592, 0.0014993064105510712, 0.013264846988022327, -0.0624217726290226, -0.0012287232093513012, 0.02897718735039234, -0.02952898107469082, -0.02569606713950634, 0.0033771232701838017, -0.028868917375802994, 1.518517706244893e-06, -0.038824815303087234, -0.019842024892568588, -0.01765807531774044, -0.008112371899187565, -0.025130528956651688, 0.02574918232858181, 0.11367510259151459, -0.006133151240646839, -0.017986435443162918, -0.045411463826894

### 3 - Armazenamento - Geração de embeddings e indexação



In [ ]:
len(chunks)

44

In [ ]:
chunks[0]

'Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital'

In [ ]:
chunks[1]

'Visão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções'

#### Armazenando no banco de dados vetorial




In [ ]:
vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

#### Salvando índice FAISS



In [ ]:
vectorstore.save_local("index_faiss")

In [ ]:
## E para carregar depois o índice salvo
#db = FAISS.load_local("index_faiss", embeddings)



## Etapas de Recuperação e geração

### 4 - Configurando o recuperador de texto / Retriever



In [ ]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d7aabd15d50>, search_kwargs={'k': 6})

### 5 - Geração



In [ ]:
prompt_rag = PromptTemplate(
    input_variables=["context", "input"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nPergunta: {input}\nContexto: {context}\n')

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [ ]:
res = chain_rag.invoke("Qual é o público alvo do banco?")
show_res(res)

De acordo com o contexto, o público-alvo do banco é:

Pessoas físicas a partir de 18 anos que buscam soluções financeiras modernas, acessíveis e com gestão 100% digital.

## Melhorias no prompt



In [ ]:
system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
Mantenha a resposta concisa.
Responda em português. \n\n"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ]
)
qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas comente que não sabe dizer com certeza.\nMas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.\nMantenha a resposta concisa.\nResponda em português. \n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Pergunta: {input}\n\n Contexto: {context}'), additional_kwargs={})])

In [ ]:
chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

res = chain_rag.invoke("Como alterar minha senha?")
show_res(res)

Para alterar sua senha, selecione "Minha conta" e "Alterar senha". O sistema exige a senha atual e, por segurança, não permite reutilizar senhas anteriores recentes.

## Melhorias no método de busca



In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3, 'fetch_k':4}
)

## Pipeline RAG avançada



### 1. Reformulação da consulta para contextualizar

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
context_q_system_prompt = context_q_system_prompt
context_q_user_prompt = "Question: {input}"
context_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", context_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", context_q_user_prompt),
    ]
)

### 2. Chain para contextualização



In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm = llm, retriever = retriever, prompt = context_q_prompt
)

### 3. Chain para perguntas e respostas (Q&A)



In [ ]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "Pergunta: {input}\n\n Contexto: {context}"),
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def config_rag_chain(llm, retriever):

    # Prompt de contextualização
    context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

    context_q_system_prompt = context_q_system_prompt
    context_q_user_prompt = "Question: {input}"
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", context_q_user_prompt),
        ]
    )

    # Chain para contextualização
    history_aware_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_q_prompt
    )

    # Prompt para perguntas e respostas (Q&A)
    system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
    Use os seguintes pedaços de contexto recuperado para responder à pergunta.
    Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
    Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
    Mantenha a resposta concisa.
    Responda em português. \n\n"""

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ])

    # Configurar LLM e Chain para perguntas e respostas (Q&A)

    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(
        history_aware_retriever,
        qa_chain,
    )

    return rag_chain

## Adicionando histórico de conversas



In [ ]:
store = {}

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [ ]:
qa_chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history"
)

In [ ]:
session_id = "usuario_42"

In [ ]:
response1 = qa_chain_with_history.invoke(
    {"input": "Como posso alterar minha senha no app?", "context": ""},
    config = {"configurable": {"session_id": session_id}}
)
print(response1)

Olá! Para alterar sua senha no app, você pode seguir os seguintes passos:

1. Abra o app e clique no ícone de perfil (geralmente localizado no canto superior direito).
2. Selecione "Configurações" ou "Perfil".
3. Encontre a opção "Senha" ou "Alterar Senha".
5. Insira a senha atual e, em seguida, a nova senha desejada.
7. Confirme a alteração.

Se você tiver alguma dúvida ou precisar de mais ajuda, por favor, me pergunte!


In [ ]:
response2 = qa_chain_with_history.invoke(
    {"input": "Qual minha última pergunta?", "context": ""},
    config={"configurable": {"session_id": session_id}},
)
print(response2)

Sua última pergunta foi "Como posso alterar minha senha no app?"


### Passando histórico de chat à nossa Chain



In [ ]:
rag_chain = config_rag_chain(llm, retriever)

In [ ]:
chat_history = []

In [ ]:
input = "como alterar minha senha?"

# 1
chat_history.append(HumanMessage(content=input))

# 2
result = rag_chain.invoke({"input": input, "chat_history": chat_history})

res = result['answer']
print(res)

# 3
chat_history.append(AIMessage(content=res))

Para alterar sua senha, selecione "Minha conta" e "Alterar senha". O sistema exige a senha atual e, por segurança, não permite reutilizar senhas anteriores recentes.


In [ ]:
chat_history

[HumanMessage(content='como alterar minha senha?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Para alterar sua senha, selecione "Minha conta" e "Alterar senha". O sistema exige a senha atual e, por segurança, não permite reutilizar senhas anteriores recentes.', additional_kwargs={}, response_metadata={})]

In [ ]:
input = "Qual foi minha última pergunta?"
result = rag_chain.invoke({
    "input": input,
    "chat_history": chat_history
})
res = result["answer"]
print(res)

Sua última pergunta foi "como alterar minha senha?"


## Configuração a função de Indexação e Recuperação


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def config_retriever(folder_path="/content"):
    # Carregar documentos
    docs_path = Path("/content")
    pdf_files = [f for f in docs_path.glob("*.pdf")]

    loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

    # Divisão em pedaços de texto / Split
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = []
    for doc in loaded_documents:
        chunks.extend(text_splitter.split_text(doc))

    # Embeddings
    embedding_model = "BAAI/bge-m3"

    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    # Armazenamento
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

    vectorstore.save_local('index_faiss')

    # Configurando o recuperador de texto / Retriever
    retriever = vectorstore.as_retriever(
        search_type='mmr',
        search_kwargs={'k':3, 'fetch_k':4}
    )

    return retriever

In [ ]:
retriever = config_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
rag_chain = config_rag_chain(llm, retriever)

## Junção da pipeline final + Conclusão

In [ ]:
def chat_llm(rag_chain, input, chat_history):
  chat_history.append(HumanMessage(content = input))

  response = rag_chain.invoke({
      "input": input,
      "chat_history": chat_history
  })

  res = response["answer"]

  chat_history.append(AIMessage(content = res))

  return chat_history

In [ ]:
chat_history = []

In [ ]:
input = "olá! como mudar minha senha?"  # @param {type:"string"}

chat_history = chat_llm(rag_chain, input, chat_history)

In [ ]:
chat_history

[HumanMessage(content='olá! qual foi minha pergunta anterior?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá! Você não fez nenhuma pergunta anterior. Essa é sua primeira pergunta!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='olá! tudo bem?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá! Tudo bem! Estou aqui para ajudar com qualquer dúvida que você tenha sobre os serviços da empresa.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='olá! como mudar minha senha?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá! Para alterar sua senha, siga os seguintes passos:\n\nNo aplicativo, clique no menu "Minha conta" e selecione "Alterar senha".\nNo site, acesse "Configurações" no menu do topo, selecione "Minha conta" e "Alterar senha".\n\nLembre-se de que o sistema exige a senha atual e, por segurança, não permite reutilizar senhas anteriores recentes.', additional_kwargs={}, response

## Interface com Streamlit







In [ ]:
!pip install -q streamlit python-dotenv
!npm install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 3s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

## Conclusão do projeto


In [ ]:
%%writefile .env
GROQ_API_KEY=#######

Writing .env


In [ ]:
%%writefile app02.py
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

st.set_page_config(page_title="Atendimento SafeBank 🤖", page_icon="🤖")
st.title("Atendimento SafeBank")

id_model = "deepseek-r1-distill-llama-70b"
temperature = 0.7
path = "/content"

### Carregamento da LLM
def load_llm(id_model, temperature):
  llm = ChatGroq(
    model=id_model,
    temperature=temperature,
    max_tokens=None,
    timeout=None,
    max_retries=2,
  )
  return llm

llm = load_llm(id_model, temperature)

### Exibição do resultado
def show_res(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()  # fallback se não houver tag
  display(Markdown(res))

### Extração do conteúdo
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()
  content = "\n".join([page.page_content for page in doc])
  return content

### Indexação e recuperação
def config_retriever(folder_path="/content"):
  # Carregar documentos
  docs_path = Path("/content")
  pdf_files = [f for f in docs_path.glob("*.pdf")]

  if len(pdf_files) < 1:
    st.error("Nenhum arquivo PDF carregado")
    st.stop()

  loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

  # Divisão em pedaços de texto / Split
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200
  )
  chunks = []
  for doc in loaded_documents:
      chunks.extend(text_splitter.split_text(doc))

  # Embeddings
  embedding_model = "BAAI/bge-m3" #sentence-transformers/all-mpnet-base-v2

  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

  # Armazenamento
  vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

  vectorstore.save_local('index_faiss')

  # Configurando o recuperador de texto / Retriever
  retriever = vectorstore.as_retriever(
      search_type='mmr',
      search_kwargs={'k':3, 'fetch_k':4}
  )

  return retriever

### Chain da RAG
def config_rag_chain(llm, retriever):

  # Prompt de contextualização
  context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

  context_q_system_prompt = context_q_system_prompt
  context_q_user_prompt = "Question: {input}"
  context_q_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", context_q_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", context_q_user_prompt),
      ]
  )

  # Chain para contextualização
  history_aware_retriever = create_history_aware_retriever(
    llm=llm, retriever=retriever, prompt=context_q_prompt
  )

  # Prompt para perguntas e respostas (Q&A)
  system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
  Use os seguintes pedaços de contexto recuperado para responder à pergunta.
  Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
  Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
  Mantenha a resposta concisa.
  Responda em português. \n\n"""

  qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "Pergunta: {input}\n\n Contexto: {context}"),
  ])

  # Configurar LLM e Chain para perguntas e respostas (Q&A)

  qa_chain = create_stuff_documents_chain(llm, qa_prompt)

  rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain,
  )

  return rag_chain

### Interação com chat
def chat_llm(rag_chain, input):

  st.session_state.chat_history.append(HumanMessage(content=input))

  response = rag_chain.invoke({
      "input": input,
      "chat_history": st.session_state.chat_history
  })

  res = response["answer"]
  res = res.split("</think>")[-1].strip() if "</think>" in res else res.strip()

  st.session_state.chat_history.append(AIMessage(content=res))

  return res

input = st.chat_input("Digite sua mensagem aqui...")

if "chat_history" not in st.session_state:
  st.session_state.chat_history = [
      AIMessage(content = "Olá, sou o seu assistente virtual! Como posso te ajudar?"),
  ]

if "retriever" not in st.session_state:
  st.session_state.retriever = None

for message in st.session_state.chat_history:
  if isinstance(message, AIMessage):
    with st.chat_message("AI"):
      st.write(message.content)
  elif isinstance(message, HumanMessage):
    with st.chat_message("Human"):
      st.write(message.content)

if input is not None:
  with st.chat_message("Human"):
    st.markdown(input)

  with st.chat_message("AI"):
    if st.session_state.retriever is None:
      st.session_state.retriever = config_retriever(path)
    rag_chain = config_rag_chain(llm, st.session_state.retriever)
    res = chat_llm(rag_chain, input)
    st.write(res)

Writing app02.py


### Execução do Streamlit

In [ ]:
!streamlit run app02.py &>/content/logs.txt &

!wget -q -O - ipv4.icanhazip.com
!npx localtunnel --port 8501

34.42.187.212
⠙your url is: https://thin-mugs-film.loca.lt
^C
